In [1]:
import os

import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from utils.constants import base_path, label_dict
from utils.parsbert import train_parsbert, test_parsbert
from utils.preprocess import clean_text

# Load raw datasets

In [2]:
pd.set_option('future.no_silent_downcasting', True)  # noqa

# Load datasets
train_df = pd.read_csv(f"{base_path}/data/train.tsv", sep="\t", header=None, names=["sentence", "label"])
test_df = pd.read_csv(f"{base_path}/data/test.tsv", sep="\t", header=None, names=["sentence", "label"])

# print distinct labels
print(train_df.label.unique())
print(test_df.label.unique())
print(set(train_df.label.unique()) == set(test_df.label.unique()))
print('*' * 50)

print(label_dict)
print(train_df.loc[0, "sentence"])
print(train_df.loc[0, "label"])
print('*' * 50)

print(train_df.dtypes)

['SAD' 'HATE' 'OTHER' 'FEAR' 'ANGRY' 'HAPPY' 'SURPRISE']
['SAD' 'HAPPY' 'OTHER' 'SURPRISE' 'FEAR' 'HATE' 'ANGRY']
True
**************************************************
{'HAPPY': 0, 'SAD': 1, 'ANGRY': 2, 'FEAR': 3, 'SURPRISE': 4, 'HATE': 5, 'OTHER': 6}
خیلی کوچیک هستن و سایزشون بدرد نمیخوره میخوام پس بدم
SAD
**************************************************
sentence    object
label       object
dtype: object


In [3]:
# clean data and save
tqdm.pandas()
if not os.path.exists(f"{base_path}/data/train_cleaned.tsv"):
    train_df['sentence'] = train_df['sentence'].progress_apply(clean_text)
    train_df.to_csv(f"{base_path}/data/train_cleaned.tsv", sep="\t", index=False)
if not os.path.exists(f"{base_path}/data/test_cleaned.tsv"):
    test_df['sentence'] = test_df['sentence'].progress_apply(clean_text)
    test_df.to_csv(f"{base_path}/data/test_cleaned.tsv", sep="\t", index=False)

# Function to apply preprocessing in parallel using joblib
# def parallel_apply(df, func):
#     processed_sentences = Parallel(n_jobs=-1)(
#         delayed(func)(text=sentence) for sentence in df['sentence'])
#     return processed_sentences
# 
# 
# if not os.path.exists(f"{base_path}/data/train_cleaned.tsv"):
#     train_df['sentence'] = parallel_apply(train_df, combined_preprocess)
# 
# if not os.path.exists(f"{base_path}/data/test_cleaned.tsv"):
#     test_df['sentence'] = parallel_apply(test_df, combined_preprocess)

# Load cleaned datasets

In [4]:
# First row is header
train_df = pd.read_csv(f"{base_path}/data/train_cleaned.tsv", sep="\t")
test_df = pd.read_csv(f"{base_path}/data/test_cleaned.tsv", sep="\t")

print(train_df.loc[0:5, "sentence"])
print(test_df.loc[0:5, "sentence"])
print(test_df.loc[6, "sentence"])

0    خیلی کوچیک هستن و سایزشون بدرد نمیخوره میخوام ...
1       از صدای پرنده دم دمای صبح متنفرم متنفرم متنفرم
2    کیفیتش خیلی خوبه با شک خریدم ولی واقعا راضیم ب...
3    چون همش با دوربین ثبت‌شده ایا میشه اعتراض زد؟؟...
4                    این وضع ب طرز خنده داری گریه داره
5    خب من رسما از یک نفر متنفرم چون از گربه بدش می...
Name: sentence, dtype: object
0    این شاید اولین عزای عمومی واقعی است که یاد دار...
1    دیشب بعد از ارسال تویت مربوط به آثار باستانی ت...
2    کدوم شعبه پول نداده بگو الان برات آمار دقیق بد...
3    امروز وسط یه بحث با بابا مامانم گفتم آدم باید ...
4    امشب گفت نامزدی دوستش که ادم روشنفکری است بهم ...
5    به امید موفقیت تیم ملی و پیروزی در بازی امروز ...
Name: sentence, dtype: object
با آرزوی موفقیت و پیروزی


In [5]:
# Split train dataset for validation
train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    train_df['sentence'], train_df['label'], test_size=0.2, random_state=42
)
print(f"Number of training sentences: {len(train_sentences)}")
print(f"Number of validation sentences: {len(val_sentences)}")
print(f"Number of test sentences: {len(test_df)}")

Number of training sentences: 4900
Number of validation sentences: 1225
Number of test sentences: 1151


# Load the tokenizer and model

In [6]:
model_names = [
    'HooshvareLab/bert-fa-base-uncased-sentiment-snappfood',  # noqa
    'HooshvareLab/bert-fa-base-uncased-sentiment-digikala',  # noqa
    'HooshvareLab/bert-fa-base-uncased',  # noqa
    'HooshvareLab/bert-fa-zwnj-base'  # ParsBERT (v3.0) # noqa
]
cache_dir = f'{base_path}/models/huggingface_cache'

# Set up the training arguments

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

for model_name in model_names:
    print(f"Model name: {model_name}")
    train_parsbert(
        model_name=model_name,
        cache_dir=cache_dir,
        device=device,
        label_dict=label_dict,
        train_sentences=train_sentences,
        train_labels=train_labels,
        val_sentences=val_sentences,
        val_labels=val_labels,
        base_path=base_path
    )

Device: cuda
Model name: HooshvareLab/bert-fa-base-uncased-sentiment-snappfood


/home/hamedhf/anaconda3/envs/DL-project/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-fa-base-uncased-sentiment-snappfood and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([7, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Training Epoch 1: 100%|█


Train Loss: 1.2156 | Train Accuracy: 0.5627


Validation Epoch 1: 100%|██████████| 154/154 [00:20<00:00,  7.34it/s]


Validation Loss: 0.8682 | Validation Accuracy: 0.6906
**************************************************
Saving new best model at epoch 1


...

Validation Loss: 1.5017 | Validation Accuracy: 0.6678
**************************************************


Training Epoch 8: 100%|██████████| 613/613 [03:55<00:00,  2.61it/s, Loss=0.00285]



Train Loss: 0.0208 | Train Accuracy: 0.9951


Validation Epoch 8: 100%|██████████| 154/154 [00:21<00:00,  7.28it/s]

Validation Loss: 1.6148 | Validation Accuracy: 0.6759
**************************************************
Early stopping at epoch 8. Best epoch: 2


In [8]:
for model_name in model_names:
    print(f"Model name: {model_name}")
    test_parsbert(
        model_name=model_name,
        cache_dir=cache_dir,
        device=device,
        label_dict=label_dict,
        test_sentences=test_df['sentence'].to_list(),
        test_labels=test_df['label'].to_list(),
        base_path=base_path
    )

Model name: HooshvareLab/bert-fa-base-uncased-sentiment-snappfood


Testing: 100%|██████████| 144/144 [00:18<00:00,  7.93it/s]


Test Loss: 1.1351 | Test Accuracy: 0.5995
Precision: 0.6524 | Recall: 0.5897 | F1 Score: 0.5936
**************************************************
Model name: HooshvareLab/bert-fa-base-uncased-sentiment-digikala


Testing: 100%|██████████| 144/144 [00:18<00:00,  7.89it/s]


Test Loss: 1.1684 | Test Accuracy: 0.5934
Precision: 0.6430 | Recall: 0.5804 | F1 Score: 0.5848
**************************************************
Model name: HooshvareLab/bert-fa-base-uncased


Testing: 100%|██████████| 144/144 [00:18<00:00,  7.91it/s]


Test Loss: 1.1109 | Test Accuracy: 0.6229
Precision: 0.6745 | Recall: 0.5994 | F1 Score: 0.6134
**************************************************
Model name: HooshvareLab/bert-fa-zwnj-base


Testing: 100%|██████████| 144/144 [00:18<00:00,  7.95it/s]

Test Loss: 1.1629 | Test Accuracy: 0.6003
Precision: 0.6630 | Recall: 0.5911 | F1 Score: 0.6030
**************************************************
